In [30]:
# HiCAR Challenge Pipeline (Updated)

import os
import random
import numpy as np
import pandas as pd
from glob import glob
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import log_loss, accuracy_score

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from PIL import Image

# Config
CFG = {
    'IMG_SIZE': 224,
    'EPOCHS': 15,
    'LR': 3e-4,
    'BATCH_SIZE': 24,
    'SEED': 2025,
    'FOLDS': 5
}

# Seed
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
set_seed(CFG['SEED'])

# Path
TRAIN_DIR = "D:/데이콘 250519 대회/clean_train"
TEST_DIR = "D:/데이콘 250519 대회/open/test"
SAMPLE_SUB = "D:/데이콘 250519 대회/open/sample_submission.csv"

# Label Mapping
label_list = sorted(os.listdir(TRAIN_DIR))
label2id = {v: i for i, v in enumerate(label_list)}
id2label = {i: v for v, i in label2id.items()}

# Dataset
class HiCarDataset(Dataset):
    def __init__(self, image_paths, labels=None, transform=None):
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img = Image.open(self.image_paths[idx]).convert("RGB")
        if self.transform:
            img = self.transform(img)

        if self.labels is not None:
            return img, self.labels[idx]
        return img

# Transform
transform = transforms.Compose([
    transforms.Resize((CFG['IMG_SIZE'], CFG['IMG_SIZE'])),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Image & Label
image_paths = glob(os.path.join(TRAIN_DIR, '*', '*.jpg'))
labels = [label2id[os.path.basename(os.path.dirname(p))] for p in image_paths]

# Model
class HiCarModel(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.backbone = models.efficientnet_b0(pretrained=True)
        in_features = self.backbone.classifier[1].in_features
        self.backbone.classifier[1] = nn.Linear(in_features, num_classes)

    def forward(self, x):
        return self.backbone(x)

# Training loop

def train_one_epoch(model, loader, optimizer, criterion, device):
    model.train()
    total_loss = 0
    for imgs, labels in loader:
        imgs, labels = imgs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(loader)


def validate_one_epoch(model, loader, criterion, device):
    model.eval()
    total_loss = 0
    preds, targets = [], []
    with torch.no_grad():
        for imgs, labels in loader:
            imgs, labels = imgs.to(device), labels.to(device)
            outputs = model(imgs)
            loss = criterion(outputs, labels)
            total_loss += loss.item()
            preds.extend(outputs.argmax(1).cpu().numpy())
            targets.extend(labels.cpu().numpy())
    acc = accuracy_score(targets, preds)
    return total_loss / len(loader), acc

# Cross-validation
skf = StratifiedKFold(n_splits=CFG['FOLDS'], shuffle=True, random_state=CFG['SEED'])
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

for fold, (train_idx, val_idx) in enumerate(skf.split(image_paths, labels)):
    print(f"Fold {fold+1}")
    train_paths = [image_paths[i] for i in train_idx]
    val_paths = [image_paths[i] for i in val_idx]
    train_labels = [labels[i] for i in train_idx]
    val_labels = [labels[i] for i in val_idx]

    train_ds = HiCarDataset(train_paths, train_labels, transform)
    val_ds = HiCarDataset(val_paths, val_labels, transform)

    train_loader = DataLoader(train_ds, batch_size=CFG['BATCH_SIZE'], shuffle=True)
    val_loader = DataLoader(val_ds, batch_size=CFG['BATCH_SIZE'], shuffle=False)

    model = HiCarModel(len(label2id)).to(device)
    optimizer = optim.AdamW(model.parameters(), lr=CFG['LR'])
    criterion = nn.CrossEntropyLoss()

    for epoch in range(CFG['EPOCHS']):
        train_loss = train_one_epoch(model, train_loader, optimizer, criterion, device)
        val_loss, val_acc = validate_one_epoch(model, val_loader, criterion, device)
        print(f"Epoch {epoch+1}: Train Loss {train_loss:.4f} | Val Loss {val_loss:.4f} | Val Acc {val_acc:.4f}")

# Inference
model.eval()
test_paths = sorted(glob(os.path.join(TEST_DIR, '*.jpg')))
test_ds = HiCarDataset(test_paths, transform=transform)
test_loader = DataLoader(test_ds, batch_size=CFG['BATCH_SIZE'], shuffle=False)

predictions = []
with torch.no_grad():
    for imgs in test_loader:
        imgs = imgs.to(device)
        outputs = model(imgs)
        probs = torch.softmax(outputs, dim=1).cpu().numpy()
        predictions.extend(probs)

# Submission
submission = pd.read_csv(SAMPLE_SUB)
submission[label2id.keys()] = predictions
submission.to_csv("submission.csv", index=False)



A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.1 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "c:\Users\user\anaconda3\envs\torch\lib\runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "c:\Users\user\anaconda3\envs\torch\lib\runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "c:\Users\user\anaconda3\envs\torch\lib\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "c:\Users\user\anaconda3\envs\torch\lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  Fil

ValueError: Cannot have number of splits n_splits=5 greater than the number of samples: n_samples=3.

In [ ]:
import tensorflow as tf
print(tf.__version__)

In [ ]:
import os
import random
import numpy as np
import pandas as pd
from glob import glob
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score

import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications.efficientnet import preprocess_input

# Config
CFG = {
    'IMG_SIZE': 224,
    'EPOCHS': 15,
    'LR': 3e-4,
    'BATCH_SIZE': 24,
    'SEED': 2025,
    'FOLDS': 5
}

# Set seed
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)

set_seed(CFG['SEED'])

# Path
TRAIN_DIR = "D:/데이콘 250519 대회/open/train"
TEST_DIR = "D:/데이콘 250519 대회/open/test"
SAMPLE_SUB = "D:/데이콘 250519 대회/open/sample_submission.csv"

# Label mapping
label_list = sorted(os.listdir(TRAIN_DIR))
label2id = {v: i for i, v in enumerate(label_list)}
id2label = {i: v for v, i in label2id.items()}

# Load data
image_paths = glob(os.path.join(TRAIN_DIR, '*', '*.jpg'))
labels = [label2id[os.path.basename(os.path.dirname(p))] for p in image_paths]

# Data preprocessing
def load_and_preprocess(img_path):
    img = load_img(img_path, target_size=(CFG['IMG_SIZE'], CFG['IMG_SIZE']))
    img = img_to_array(img)
    img = preprocess_input(img)
    return img

# TF Dataset 생성 함수
def create_dataset(image_paths, labels=None, is_train=True):
    def gen():
        for i, path in enumerate(image_paths):
            img = load_and_preprocess(path)
            if labels is not None:
                yield img, labels[i]
            else:
                yield img

    if labels is not None:
        ds = tf.data.Dataset.from_generator(
            gen,
            output_types=(tf.float32, tf.int32),
            output_shapes=((CFG['IMG_SIZE'], CFG['IMG_SIZE'], 3), ())
        )
    else:
        ds = tf.data.Dataset.from_generator(
            gen,
            output_types=tf.float32,
            output_shapes=(CFG['IMG_SIZE'], CFG['IMG_SIZE'], 3)
        )

    if is_train:
        ds = ds.shuffle(1024)
    ds = ds.batch(CFG['BATCH_SIZE']).prefetch(tf.data.AUTOTUNE)
    return ds

# Model
def build_model(num_classes):
    base = tf.keras.applications.EfficientNetB0(include_top=False, input_shape=(CFG['IMG_SIZE'], CFG['IMG_SIZE'], 3), weights='imagenet', pooling='avg')
    x = layers.Dense(num_classes, activation='softmax')(base.output)
    model = models.Model(inputs=base.input, outputs=x)
    return model

# Cross-validation training
skf = StratifiedKFold(n_splits=CFG['FOLDS'], shuffle=True, random_state=CFG['SEED'])

for fold, (train_idx, val_idx) in enumerate(skf.split(image_paths, labels)):
    print(f"\n### Fold {fold+1}")

    train_paths = [image_paths[i] for i in train_idx]
    val_paths = [image_paths[i] for i in val_idx]
    train_labels = [labels[i] for i in train_idx]
    val_labels = [labels[i] for i in val_idx]

    train_ds = create_dataset(train_paths, train_labels, is_train=True)
    val_ds = create_dataset(val_paths, val_labels, is_train=False)

    model = build_model(num_classes=len(label2id))
    model.compile(optimizer=optimizers.Adam(CFG['LR']),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    model.fit(train_ds,
              validation_data=val_ds,
              epochs=CFG['EPOCHS'],
              verbose=1)

# Inference
test_paths = sorted(glob(os.path.join(TEST_DIR, '*.jpg')))
test_ds = create_dataset(test_paths, is_train=False)

preds = model.predict(test_ds)
print("Inference 완료")

# Submission
submission = pd.read_csv(SAMPLE_SUB)
for idx, class_name in enumerate(label2id.keys()):
    submission[class_name] = preds[:, idx]

submission.to_csv("submission.csv", index=False)
print("submission.csv 저장 완료")



### Fold 1
Epoch 1/15
1105/1105 [==============================] - 523s 241ms/step - loss: 2.6151 - accuracy: 0.5248 - val_loss: 4.1993 - val_accuracy: 0.2067
Epoch 2/15
1105/1105 [==============================] - 147s 126ms/step - loss: 0.7194 - accuracy: 0.8166 - val_loss: 2.7278 - val_accuracy: 0.3585
Epoch 3/15
1105/1105 [==============================] - 99s 87ms/step - loss: 0.3196 - accuracy: 0.9160 - val_loss: 2.4004 - val_accuracy: 0.4262
Epoch 4/15
1105/1105 [==============================] - 97s 86ms/step - loss: 0.2041 - accuracy: 0.9457 - val_loss: 2.2646 - val_accuracy: 0.4730
Epoch 5/15
1105/1105 [==============================] - 97s 86ms/step - loss: 0.1533 - accuracy: 0.9597 - val_loss: 2.0792 - val_accuracy: 0.5023
Epoch 6/15
1105/1105 [==============================] - 97s 86ms/step - loss: 0.1198 - accuracy: 0.9685 - val_loss: 1.7726 - val_accuracy: 0.5821
Epoch 7/15
1105/1105 [==============================] - 96s 85ms/step - loss: 0.0967 - accuracy: 0.9751 - va

In [ ]:
import os
import shutil
from PIL import Image
from tqdm import tqdm
import numpy as np

# 원본 경로
TRAIN_DIR = "D:/데이콘 250519 대회/open/train"
FILTERED_TRAIN_DIR = "D:/데이콘 250519 대회/open/filtered_train"
OTHERS_DIR = "D:/데이콘 250519 대회/open/others"

os.makedirs(FILTERED_TRAIN_DIR, exist_ok=True)
os.makedirs(OTHERS_DIR, exist_ok=True)


In [ ]:
def is_full_car_image(image_path):
    """
    간단한 기준으로 차 외관 전체가 있는지 판별하는 함수 (크기 + 색상 기준)
    """
    try:
        img = Image.open(image_path).convert("RGB")
        w, h = img.size

        # 외관 기준: 세로 또는 가로가 너무 작으면 제외
        if w < 150 or h < 150:
            return False

        # 전체적으로 너무 어둡거나 너무 밝은 이미지는 제외 (내부 이미지 등)
        img_np = np.array(img.resize((64, 64))) / 255.0
        mean_brightness = img_np.mean()

        if mean_brightness < 0.2 or mean_brightness > 0.9:
            return False

        return True
    except:
        return False


In [ ]:
from glob import glob

car_classes = sorted(os.listdir(TRAIN_DIR))
for class_name in tqdm(car_classes):
    class_dir = os.path.join(TRAIN_DIR, class_name)
    save_dir = os.path.join(FILTERED_TRAIN_DIR, class_name)
    os.makedirs(save_dir, exist_ok=True)

    image_paths = glob(os.path.join(class_dir, '*.jpg'))

    for img_path in image_paths:
        if is_full_car_image(img_path):
            shutil.copy(img_path, os.path.join(save_dir, os.path.basename(img_path)))
        else:
            shutil.copy(img_path, os.path.join(OTHERS_DIR, f"{class_name}_{os.path.basename(img_path)}"))


100%|██████████| 396/396 [03:13<00:00,  2.04it/s]


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications.efficientnet import EfficientNetB0, preprocess_input
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
import random
import pandas as pd

# Config
CFG = {
    'IMG_SIZE': 224,
    'EPOCHS': 15,
    'LR': 3e-4,
    'BATCH_SIZE': 24,
    'SEED': 2025,
    'FOLDS': 5
}

# Seed
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)
set_seed(CFG['SEED'])

# Label mapping
label_list = sorted(os.listdir(FILTERED_TRAIN_DIR))
label2id = {v: i for i, v in enumerate(label_list)}
id2label = {i: v for v, i in label2id.items()}

# Load filtered data
from glob import glob

image_paths = glob(os.path.join(FILTERED_TRAIN_DIR, '*', '*.jpg'))
labels = [label2id[os.path.basename(os.path.dirname(p))] for p in image_paths]

# DataLoader
def load_and_preprocess(img_path):
    img = load_img(img_path, target_size=(CFG['IMG_SIZE'], CFG['IMG_SIZE']))
    img = img_to_array(img)
    img = preprocess_input(img)
    return img

def create_dataset(image_paths, labels=None, is_train=True):
    def gen():
        for i, path in enumerate(image_paths):
            img = load_and_preprocess(path)
            if labels is not None:
                yield img, labels[i]
            else:
                yield img

    if labels is not None:
        ds = tf.data.Dataset.from_generator(
            gen,
            output_types=(tf.float32, tf.int32),
            output_shapes=((CFG['IMG_SIZE'], CFG['IMG_SIZE'], 3), ())
        )
    else:
        ds = tf.data.Dataset.from_generator(
            gen,
            output_types=tf.float32,
            output_shapes=(CFG['IMG_SIZE'], CFG['IMG_SIZE'], 3)
        )

    if is_train:
        ds = ds.shuffle(1024)
    ds = ds.batch(CFG['BATCH_SIZE']).prefetch(tf.data.AUTOTUNE)
    return ds



In [ ]:
def build_model(num_classes):
    base = EfficientNetB0(include_top=False, input_shape=(CFG['IMG_SIZE'], CFG['IMG_SIZE'], 3), weights='imagenet', pooling='avg')
    x = layers.Dense(128, activation='relu')(base.output)
    x = layers.Dropout(0.4)(x)
    output = layers.Dense(num_classes, activation='softmax')(x)
    model = models.Model(inputs=base.input, outputs=output)
    return model

# Stratified K-Fold
skf = StratifiedKFold(n_splits=CFG['FOLDS'], shuffle=True, random_state=CFG['SEED'])

for fold, (train_idx, val_idx) in enumerate(skf.split(image_paths, labels)):
    print(f"\n### Fold {fold+1}")

    train_paths = [image_paths[i] for i in train_idx]
    val_paths = [image_paths[i] for i in val_idx]
    train_labels = [labels[i] for i in train_idx]
    val_labels = [labels[i] for i in val_idx]

    train_ds = create_dataset(train_paths, train_labels, is_train=True)
    val_ds = create_dataset(val_paths, val_labels, is_train=False)

    model = build_model(len(label2id))
    model.compile(optimizer=optimizers.Adam(CFG['LR']),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    model.fit(train_ds,
              validation_data=val_ds,
              epochs=CFG['EPOCHS'],
              callbacks=[
                  tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True),
                  tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3)
              ])



### Fold 1
Epoch 1/15
1104/1104 [==============================] - 425s 204ms/step - loss: 5.6717 - accuracy: 0.0250 - val_loss: 6.1473 - val_accuracy: 0.0026 - lr: 3.0000e-04
Epoch 2/15
1104/1104 [==============================] - 173s 150ms/step - loss: 4.3127 - accuracy: 0.1564 - val_loss: 6.7292 - val_accuracy: 0.0104 - lr: 3.0000e-04
Epoch 3/15
1104/1104 [==============================] - 100s 88ms/step - loss: 2.8119 - accuracy: 0.3653 - val_loss: 7.0519 - val_accuracy: 0.0633 - lr: 3.0000e-04
Epoch 4/15
1104/1104 [==============================] - 111s 99ms/step - loss: 1.7504 - accuracy: 0.5597 - val_loss: 7.0581 - val_accuracy: 0.1453 - lr: 3.0000e-04
Epoch 5/15
1104/1104 [==============================] - 98s 87ms/step - loss: 1.1885 - accuracy: 0.6825 - val_loss: 6.0643 - val_accuracy: 0.2151 - lr: 1.5000e-04
Epoch 6/15
1104/1104 [==============================] - 98s 87ms/step - loss: 0.8607 - accuracy: 0.7660 - val_loss: 5.8893 - val_accuracy: 0.2665 - lr: 1.5000e-04
Epoc

In [3]:
TEST_DIR = "D:/데이콘 250519 대회/open/test"
SAMPLE_SUB = "D:/데이콘 250519 대회/open/sample_submission.csv"

test_paths = sorted(glob(os.path.join(TEST_DIR, '*.jpg')))
test_ds = create_dataset(test_paths, is_train=False)

preds = model.predict(test_ds)

# Submission
submission = pd.read_csv(SAMPLE_SUB)
for idx, class_name in enumerate(label2id.keys()):
    submission[class_name] = preds[:, idx]
submission.to_csv("submission.csv", index=False)
print("submission2.csv 저장 완료")


345/345 [==============================] - 24s 69ms/step
submission2.csv 저장 완료


In [ ]:
!pip install git+https://github.com/openai/CLIP.git


  Cloning https://github.com/openai/CLIP.git to c:\users\user\appdata\local\temp\pip-req-build-gvcjf3ps
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for clip: filename=clip-1.0-py3-none-any.whl size=1369632 sha256=cbc231f6fd1c4584bd6a0d413fe4fd071960c878b30d2a47de2b4196364cd836
  Stored in directory: C:\Users\user\AppData\Local\Temp\pip-ephem-wheel-cache-n990sk8s\wheels\c8\e4\e1\11374c111387672fc2068dfbe0d4b424cb9cdd1b2e184a71b5
Successfully built clip


  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git 'C:\Users\user\AppData\Local\Temp\pip-req-build-gvcjf3ps'
  DEPRECATION: Building 'clip' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'clip'. Discussion can be found at https://github.com/pypa/pip/issues/6334


In [ ]:
# clip_filter.py
import os
import shutil
from PIL import Image
import torch
import clip
from tqdm import tqdm

# Config
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
IMAGE_DIR = "D:/데이콘 250519 대회/open/train"
CLEAN_DIR = "D:/데이콘 250519 대회/clean_train"
FILTERED_DIR = "D:/데이콘 250519 대회/filtered_train"
THRESHOLD = 0.30  # CLIP 유사도 기준값

# Load CLIP
model, preprocess = clip.load("ViT-B/32", device=DEVICE)

# Prompt
text_inputs = clip.tokenize(["a car with full exterior view"]).to(DEVICE)
with torch.no_grad():
    text_features = model.encode_text(text_inputs)
    text_features /= text_features.norm(dim=-1, keepdim=True)

# 폴더 생성
for base_dir in [CLEAN_DIR, FILTERED_DIR]:
    for label in os.listdir(IMAGE_DIR):
        os.makedirs(os.path.join(base_dir, label), exist_ok=True)

# 필터링
for label in tqdm(os.listdir(IMAGE_DIR), desc="Filtering"):
    img_dir = os.path.join(IMAGE_DIR, label)
    for img_name in os.listdir(img_dir):
        img_path = os.path.join(img_dir, img_name)

        try:
            image = preprocess(Image.open(img_path).convert("RGB")).unsqueeze(0).to(DEVICE)
            with torch.no_grad():
                image_features = model.encode_image(image)
                image_features /= image_features.norm(dim=-1, keepdim=True)
                similarity = (image_features @ text_features.T).squeeze().item()

            if similarity >= THRESHOLD:
                shutil.copy(img_path, os.path.join(CLEAN_DIR, label, img_name))
            else:
                shutil.copy(img_path, os.path.join(FILTERED_DIR, label, img_name))

        except Exception as e:
            print(f"[ERROR] {img_path}: {e}")


100%|███████████████████████████████████████| 338M/338M [00:30<00:00, 11.5MiB/s]
Filtering: 100%|██████████| 396/396 [35:37<00:00,  5.40s/it]


In [ ]:
import os
import random
import numpy as np
import pandas as pd
from glob import glob
from sklearn.model_selection import StratifiedKFold
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications.efficientnet import preprocess_input

# Config
CFG = {
    'IMG_SIZE': 224,
    'EPOCHS': 15,
    'LR': 3e-4,
    'BATCH_SIZE': 24,
    'SEED': 2025,
    'FOLDS': 5
}

# Seed 고정
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)

set_seed(CFG['SEED'])

# 경로
TRAIN_DIR = "D:/데이콘 250519 대회/clean_train"  # 필터링된 clean_train 사용
TEST_DIR = "D:/데이콘 250519 대회/open/test"
SAMPLE_SUB = "D:/데이콘 250519 대회/open/sample_submission.csv"

# 라벨 매핑
label_list = sorted(os.listdir(TRAIN_DIR))
label2id = {v: i for i, v in enumerate(label_list)}
id2label = {i: v for v, i in label2id.items()}

# 데이터 로드
image_paths = glob(os.path.join(TRAIN_DIR, '*', '*.jpg'))
labels = [label2id[os.path.basename(os.path.dirname(p))] for p in image_paths]

# 데이터 개수 및 Fold 수 출력 및 조정
print(f"총 이미지 수: {len(image_paths)}")
print(f"Fold 수 (기본값): {CFG['FOLDS']}")

if len(image_paths) < CFG['FOLDS']:
    print(f"데이터 수({len(image_paths)})가 Fold 수({CFG['FOLDS']})보다 적어 Fold 수를 조정합니다.")
    CFG['FOLDS'] = len(image_paths)

print(f"최종 Fold 수: {CFG['FOLDS']}")

# 이미지 전처리
def load_and_preprocess(img_path):
    img = load_img(img_path, target_size=(CFG['IMG_SIZE'], CFG['IMG_SIZE']))
    img = img_to_array(img)
    img = preprocess_input(img)
    return img

# TF Dataset 생성
def create_dataset(image_paths, labels=None, is_train=True):
    def gen():
        for i, path in enumerate(image_paths):
            img = load_and_preprocess(path)
            if labels is not None:
                yield img, labels[i]
            else:
                yield img

    if labels is not None:
        ds = tf.data.Dataset.from_generator(
            gen,
            output_types=(tf.float32, tf.int32),
            output_shapes=((CFG['IMG_SIZE'], CFG['IMG_SIZE'], 3), ())
        )
    else:
        ds = tf.data.Dataset.from_generator(
            gen,
            output_types=tf.float32,
            output_shapes=(CFG['IMG_SIZE'], CFG['IMG_SIZE'], 3)
        )

    if is_train:
        ds = ds.shuffle(1024)
    ds = ds.batch(CFG['BATCH_SIZE']).prefetch(tf.data.AUTOTUNE)
    return ds

# 모델 정의
def build_model(num_classes):
    base = tf.keras.applications.EfficientNetB0(
        include_top=False, input_shape=(CFG['IMG_SIZE'], CFG['IMG_SIZE'], 3), weights='imagenet', pooling='avg')
    x = layers.Dense(num_classes, activation='softmax')(base.output)
    model = models.Model(inputs=base.input, outputs=x)
    return model

# 교차검증 학습
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedKFold

skf = StratifiedKFold(n_splits=CFG['FOLDS'], shuffle=True, random_state=CFG['SEED'])

for fold, (train_idx, val_idx) in enumerate(skf.split(image_paths, labels)):
    print(f"\n### Fold {fold+1}")

    train_paths = [image_paths[i] for i in train_idx]
    val_paths = [image_paths[i] for i in val_idx]
    train_labels = [labels[i] for i in train_idx]
    val_labels = [labels[i] for i in val_idx]

    train_ds = create_dataset(train_paths, train_labels, is_train=True)
    val_ds = create_dataset(val_paths, val_labels, is_train=False)

    model = build_model(num_classes=len(label2id))
    model.compile(optimizer=optimizers.Adam(CFG['LR']),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    model.fit(train_ds,
              validation_data=val_ds,
              epochs=CFG['EPOCHS'],
              verbose=1)

# Inference
test_paths = sorted(glob(os.path.join(TEST_DIR, '*.jpg')))
test_ds = create_dataset(test_paths, is_train=False)
preds = model.predict(test_ds)
print("✅ Inference 완료")

# 제출 파일 생성
submission = pd.read_csv(SAMPLE_SUB)
for idx, class_name in enumerate(label2id.keys()):
    submission[class_name] = preds[:, idx]

submission.to_csv("submission.csv", index=False)
print("✅ submission.csv 저장 완료")



총 이미지 수: 3
Fold 수 (기본값): 5
데이터 수(3)가 Fold 수(5)보다 적어 Fold 수를 조정합니다.
최종 Fold 수: 3


ValueError: n_splits=3 cannot be greater than the number of members in each class.

In [ ]:
import os
import random
import shutil
import numpy as np
import pandas as pd
from glob import glob
from tqdm import tqdm
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score

import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications.efficientnet import preprocess_input

import torch
import clip
from PIL import Image

# Config
CFG = {
    'IMG_SIZE': 224,
    'EPOCHS': 15,
    'LR': 3e-4,
    'BATCH_SIZE': 24,
    'SEED': 2025,
    'FOLDS': 5
}

# Set seed
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)

set_seed(CFG['SEED'])

# Paths
TRAIN_DIR = "D:/데이콘 250519 대회/open/train"
TEST_DIR = "D:/데이콘 250519 대회/open/test"
SAMPLE_SUB = "D:/데이콘 250519 대회/open/sample_submission.csv"
FILTERED_DIR = "D:/데이콘 250519 대회/open/filtered_train"

os.makedirs(FILTERED_DIR, exist_ok=True)

# CLIP filtering
device = "cuda" if torch.cuda.is_available() else "cpu"
clip_model, clip_preprocess = clip.load("ViT-B/32", device=device)

prompt_texts = ["a full car image", "only license plate", "interior of a car", "partial car image"]
text_tokens = clip.tokenize(prompt_texts).to(device)
with torch.no_grad():
    text_features = clip_model.encode_text(text_tokens)

filtered = []

for folder in os.listdir(TRAIN_DIR):
    cls_dir = os.path.join(TRAIN_DIR, folder)
    filtered_cls_dir = os.path.join(FILTERED_DIR, folder)
    os.makedirs(filtered_cls_dir, exist_ok=True)

    for img_file in os.listdir(cls_dir):
        img_path = os.path.join(cls_dir, img_file)
        image = clip_preprocess(Image.open(img_path)).unsqueeze(0).to(device)

        with torch.no_grad():
            image_features = clip_model.encode_image(image)
            similarity = (100.0 * image_features @ text_features.T).softmax(dim=-1)

        full_prob = similarity[0][0].item()
        if full_prob >= 0.5:
            shutil.copy(img_path, os.path.join(filtered_cls_dir, img_file))
        else:
            filtered.append(img_path)

print(f"Filtered out {len(filtered)} images.")

# Label mapping
label_list = sorted(os.listdir(FILTERED_DIR))
label2id = {v: i for i, v in enumerate(label_list)}
id2label = {i: v for v, i in label2id.items()}

# Load data
image_paths = glob(os.path.join(FILTERED_DIR, '*', '*.jpg'))
labels = [label2id[os.path.basename(os.path.dirname(p))] for p in image_paths]

# Load & preprocess
def load_and_preprocess(img_path):
    img = load_img(img_path, target_size=(CFG['IMG_SIZE'], CFG['IMG_SIZE']))
    img = img_to_array(img)
    img = preprocess_input(img)
    return img

# TF Dataset 생성
def create_dataset(image_paths, labels=None, is_train=True):
    def gen():
        for i, path in enumerate(image_paths):
            img = load_and_preprocess(path)
            if labels is not None:
                yield img, labels[i]
            else:
                yield img

    if labels is not None:
        ds = tf.data.Dataset.from_generator(
            gen,
            output_types=(tf.float32, tf.int32),
            output_shapes=((CFG['IMG_SIZE'], CFG['IMG_SIZE'], 3), ())
        )
        if is_train:
            ds = ds.shuffle(1024)
        ds = ds.map(augment_image)
    else:
        ds = tf.data.Dataset.from_generator(
            gen,
            output_types=tf.float32,
            output_shapes=(CFG['IMG_SIZE'], CFG['IMG_SIZE'], 3)
        )
    ds = ds.batch(CFG['BATCH_SIZE']).prefetch(tf.data.AUTOTUNE)
    return ds

# Data augmentation
def augment_image(image, label):
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_brightness(image, max_delta=0.1)
    image = tf.image.random_contrast(image, 0.8, 1.2)
    return image, label

# Model
def build_model(num_classes):
    base = tf.keras.applications.EfficientNetB0(include_top=False,
                                                input_shape=(CFG['IMG_SIZE'], CFG['IMG_SIZE'], 3),
                                                weights='imagenet',
                                                pooling='avg')
    x = layers.Dropout(0.3)(base.output)
    x = layers.Dense(num_classes, activation='softmax')(x)
    model = models.Model(inputs=base.input, outputs=x)
    return model

# Cross-validation
skf = StratifiedKFold(n_splits=CFG['FOLDS'], shuffle=True, random_state=CFG['SEED'])

for fold, (train_idx, val_idx) in enumerate(skf.split(image_paths, labels)):
    print(f"\n### Fold {fold+1}")

    train_paths = [image_paths[i] for i in train_idx]
    val_paths = [image_paths[i] for i in val_idx]
    train_labels = [labels[i] for i in train_idx]
    val_labels = [labels[i] for i in val_idx]

    train_ds = create_dataset(train_paths, train_labels, is_train=True)
    val_ds = create_dataset(val_paths, val_labels, is_train=False)

    model = build_model(num_classes=len(label2id))
    model.compile(optimizer=optimizers.Adam(CFG['LR']),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    # Callbacks
    callbacks = [
        tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=4, restore_best_weights=True),
        tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, verbose=1)
    ]

    model.fit(train_ds,
              validation_data=val_ds,
              epochs=CFG['EPOCHS'],
              callbacks=callbacks,
              verbose=1)

# Inference
test_paths = sorted(glob(os.path.join(TEST_DIR, '*.jpg')))
test_ds = create_dataset(test_paths, is_train=False)

preds = model.predict(test_ds)
print("Inference 완료")

# Submission
submission = pd.read_csv(SAMPLE_SUB)
for idx, class_name in enumerate(label2id.keys()):
    submission[class_name] = preds[:, idx]

submission.to_csv("submission.csv", index=False)
print("submission4.csv 저장 완료")


ValueError: Name tf.RaggedTensorSpec has already been registered for class tensorflow.python.ops.ragged.ragged_tensor.RaggedTensorSpec.

In [ ]:
!pip install opt_einsum

  Using cached opt_einsum-3.4.0-py3-none-any.whl.metadata (6.3 kB)
Using cached opt_einsum-3.4.0-py3-none-any.whl (71 kB)


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.19.0 requires flatbuffers>=24.3.25, which is not installed.
tensorflow 2.19.0 requires google-pasta>=0.1.1, which is not installed.
tensorflow 2.19.0 requires grpcio<2.0,>=1.24.3, which is not installed.
tensorflow 2.19.0 requires h5py>=3.11.0, which is not installed.
tensorflow 2.19.0 requires keras>=3.5.0, which is not installed.
tensorflow 2.19.0 requires libclang>=13.0.0, which is not installed.
tensorflow 2.19.0 requires tensorboard~=2.19.0, which is not installed.
tensorflow 2.19.0 requires tensorflow-io-gcs-filesystem>=0.23.1; python_version < "3.12", which is not installed.
tensorflow 2.19.0 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<6.0.0dev,>=3.20.3, but you have protobuf 6.31.0 which is incompatible.


In [1]:
import os
import random
import numpy as np
import pandas as pd
from glob import glob
from tqdm import tqdm
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score

import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications.efficientnet import preprocess_input

# Config
CFG = {
    'IMG_SIZE': 224,
    'EPOCHS': 15,
    'LR': 3e-4,
    'BATCH_SIZE': 24,
    'SEED': 2025,
    'FOLDS': 5
}

# Set seed
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)

set_seed(CFG['SEED'])

# Paths
TRAIN_DIR = "D:/데이콘 250519 대회/open/train"
TEST_DIR = "D:/데이콘 250519 대회/open/test"
SAMPLE_SUB = "D:/데이콘 250519 대회/open/sample_submission.csv"

# Label mapping
label_list = sorted(os.listdir(TRAIN_DIR))
label2id = {v: i for i, v in enumerate(label_list)}
id2label = {i: v for v, i in label2id.items()}

# Load image paths and labels
image_paths = glob(os.path.join(TRAIN_DIR, '*', '*.jpg'))
labels = [label2id[os.path.basename(os.path.dirname(p))] for p in image_paths]

# Load & preprocess
def load_and_preprocess(img_path):
    img = load_img(img_path, target_size=(CFG['IMG_SIZE'], CFG['IMG_SIZE']))
    img = img_to_array(img)
    img = preprocess_input(img)
    return img

# Data augmentation
def augment_image(image, label):
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_brightness(image, max_delta=0.1)
    image = tf.image.random_contrast(image, 0.8, 1.2)
    return image, label

# TF Dataset 생성
def create_dataset(image_paths, labels=None, is_train=True):
    def gen():
        for i, path in enumerate(image_paths):
            img = load_and_preprocess(path)
            if labels is not None:
                yield img, labels[i]
            else:
                yield img

    if labels is not None:
        ds = tf.data.Dataset.from_generator(
            gen,
            output_types=(tf.float32, tf.int32),
            output_shapes=((CFG['IMG_SIZE'], CFG['IMG_SIZE'], 3), ())
        )
        if is_train:
            ds = ds.shuffle(1024)
        ds = ds.map(augment_image)
    else:
        ds = tf.data.Dataset.from_generator(
            gen,
            output_types=tf.float32,
            output_shapes=(CFG['IMG_SIZE'], CFG['IMG_SIZE'], 3)
        )
    ds = ds.batch(CFG['BATCH_SIZE']).prefetch(tf.data.AUTOTUNE)
    return ds

# Model
def build_model(num_classes):
    base = tf.keras.applications.EfficientNetB0(include_top=False,
                                                input_shape=(CFG['IMG_SIZE'], CFG['IMG_SIZE'], 3),
                                                weights='imagenet',
                                                pooling='avg')
    x = layers.Dropout(0.3)(base.output)
    x = layers.Dense(num_classes, activation='softmax')(x)
    model = models.Model(inputs=base.input, outputs=x)
    return model

# Cross-validation training
skf = StratifiedKFold(n_splits=CFG['FOLDS'], shuffle=True, random_state=CFG['SEED'])

for fold, (train_idx, val_idx) in enumerate(skf.split(image_paths, labels)):
    print(f"\n### Fold {fold+1}")

    train_paths = [image_paths[i] for i in train_idx]
    val_paths = [image_paths[i] for i in val_idx]
    train_labels = [labels[i] for i in train_idx]
    val_labels = [labels[i] for i in val_idx]

    train_ds = create_dataset(train_paths, train_labels, is_train=True)
    val_ds = create_dataset(val_paths, val_labels, is_train=False)

    model = build_model(num_classes=len(label2id))
    model.compile(optimizer=optimizers.Adam(CFG['LR']),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    # Callbacks
    callbacks = [
        tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=4, restore_best_weights=True),
        tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, verbose=1)
    ]

    model.fit(train_ds,
              validation_data=val_ds,
              epochs=CFG['EPOCHS'],
              callbacks=callbacks,
              verbose=1)

# Inference
test_paths = sorted(glob(os.path.join(TEST_DIR, '*.jpg')))
test_ds = create_dataset(test_paths, is_train=False)

preds = model.predict(test_ds)
print("Inference 완료")

# Submission
submission = pd.read_csv(SAMPLE_SUB)
for idx, class_name in enumerate(label2id.keys()):
    submission[class_name] = preds[:, idx]

submission.to_csv("submission.csv", index=False)
print("submission.csv 저장 완료")



### Fold 1
Epoch 1/15
1105/1105 [==============================] - 345s 91ms/step - loss: 2.8977 - accuracy: 0.4722 - val_loss: 4.5811 - val_accuracy: 0.1734 - lr: 3.0000e-04
Epoch 2/15
1105/1105 [==============================] - 101s 89ms/step - loss: 0.8868 - accuracy: 0.7816 - val_loss: 2.9406 - val_accuracy: 0.3300 - lr: 3.0000e-04
Epoch 3/15
1105/1105 [==============================] - 99s 87ms/step - loss: 0.3926 - accuracy: 0.8966 - val_loss: 2.3576 - val_accuracy: 0.4244 - lr: 3.0000e-04
Epoch 4/15
1105/1105 [==============================] - 100s 88ms/step - loss: 0.2459 - accuracy: 0.9370 - val_loss: 2.3779 - val_accuracy: 0.4510 - lr: 3.0000e-04
Epoch 5/15
1105/1105 [==============================] - 98s 87ms/step - loss: 0.1810 - accuracy: 0.9514 - val_loss: 2.0148 - val_accuracy: 0.5199 - lr: 3.0000e-04
Epoch 6/15
1105/1105 [==============================] - 98s 87ms/step - loss: 0.1373 - accuracy: 0.9642 - val_loss: 2.0262 - val_accuracy: 0.5400 - lr: 3.0000e-04
Epoch 7